In [2]:
import os
import pickle
import PIL.Image
import numpy as np
import pandas as pd
import csv
import sys
from pathlib import Path
import copy

import matplotlib.pyplot as plt
%matplotlib inline

# path to stylegan encoder on server
sys.path.insert(1, '/home/shavit/APhotoADay/data/stylegan-encoder')

from encoder.generator_model import Generator
import dnnlib
import dnnlib.tflib as tflib
import config

from IPython.display import Markdown, display

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn import preprocessing

from tensorflow import keras

/home/shavit/miniconda3/envs/myenv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/shavit/miniconda3/envs/myenv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/shavit/miniconda3/envs/myenv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/shavit/miniconda3/envs/myenv/lib/python3

In [3]:
URL_FFHQ = 'https://drive.google.com/uc?id=1MEGjdvVpUsu1jB4zrXZN7Y4kBBOzizDQ'

tflib.init_tf()
with dnnlib.util.open_url(URL_FFHQ, cache_dir=config.cache_dir) as f:
    generator_network, discriminator_network, Gs_network = pickle.load(f)

generator = Generator(Gs_network, batch_size=1, randomize_noise=False)



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [10]:
import numpy as np

#Insert Puzer's direction here
puzerDirection = np.load("age.npy").flatten()

def puzerInterpolation (startPhoto, alpha):
    return startPhoto + puzerDirection * alpha

# startPhoto is npy of first photo
# frames is the number of frames wanted
# alpha is how much to age values above 0 deage, values bellow 0 age
# not clear how much alpha leads to how much aging
# try playing around with it but i think -0.2 ~ 1 is the range we want
# returns frames of the aging video in order
def createVideo (startPhoto, frames, alpha):
    video = np.empty ([frames, 512 * 18], dtype = np.float64)
    for i in range(frames):
        video[i] = puzerInterpolation(startPhoto, i * alpha / frames)

    return video

In [11]:
def generate_image(latent_vector, size):
    latent_vector = latent_vector.reshape((1, 18, 512))
    generator.set_dlatents(latent_vector)
    img_array = generator.generate_images()[0]
    img = PIL.Image.fromarray(img_array, 'RGB')
    return img.resize((size, size))

In [23]:
os.chdir("/home/shavit/APhotoADay/True_vs_Learned/")

init_p = np.load("regression_data/test/kobi.npy").flatten()
results_path = "puzerEQ/kobi10full"

if not os.path.exists(results_path):
    os.mkdir(results_path)

video = createVideo(init_p, 700, -10)

frames = 0
for i, prediction in enumerate(video):
    img = generate_image(prediction, 512)
    img.save(results_path + "/" + str(i) + ".png")
    frames = i

print(frames)

699


In [24]:
import math
framerate = math.ceil(frames / 6.0)

In [25]:
# ffmpeg -f image2 -framerate 119 -pattern_type sequence -start_number 0 -r 2 -i %d.png -vcodec libx264 -acodec aac real_broken.mp4
# ffmpeg -i real_broken.mp4 -c:v libx264 -profile:v baseline -level 3.0 -pix_fmt yuv420p real.mp4
# ffmpeg -itsscale 0.027 -i real.mp4 -c copy real_output.mp4
# os.getcwd()
# First, manually check current path and if it is a father folder, execute chdir 
os.chdir("/home/shavit/APhotoADay/True_vs_Learned/" + results_path)

res1 = os.system("ffmpeg -f image2 -framerate " + str(framerate) + " -pattern_type sequence -start_number 0 -r 2 -i %d.png -vcodec libx264 -acodec aac real_broken.mp4")

if res1 == 0:
    res2 =os.system("ffmpeg -i real_broken.mp4 -c:v libx264 -profile:v baseline -level 3.0 -pix_fmt yuv420p real.mp4")
    
    if res2 == 0:
        res3 = os.system("ffmpeg -itsscale 0.027 -i real.mp4 -c copy output.mp4")
        
        if res3 == 0:
            os.system("rm real_broken.mp4")
            os.system("rm real.mp4")